In [23]:
'''
01_download-data.ipynb
'''

import urllib.request
import pathlib
import pathlib
import os
import subprocess
import multiprocessing

import geopandas as gpd
import pandas as pd

num_cores = multiprocessing.cpu_count()

PATH_ROOT = os.path.join(pathlib.Path().absolute(), '../..' )
PATH_DATA = PATH_ROOT + '/data/01_raw/'

# links
LINK_NE_RIVERS = "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/physical/ne_10m_rivers_lake_centerlines_scale_rank.zip"
LINK_NE_RIVERS_EU = "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/physical/ne_10m_rivers_europe.zip"

LINK_SRTM_DIR = 'https://dds.cr.usgs.gov/srtm/version2_1/SRTM3/Eurasia/'

DIR_OUT_SRTM  = PATH_DATA + 'srtm/'
DIR_OUT_RIVERS  = PATH_DATA + 'rivers/'

# create folders if they do not exists
os.makedirs(DIR_OUT_SRTM) if not os.path.exists(DIR_OUT_SRTM) else False
os.makedirs(DIR_OUT_RIVERS) if not os.path.exists(DIR_OUT_RIVERS)  else False

False

In [12]:
# download google spreadsheet tables

LINK_ACTIVITIES = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSKbKoMl477qDAiyhj9sT9TZlGC7OGzShbDAaELXgkPSR95_CkmZsWmN3vLGsJgEBDkiLdn4Be_mkBJ/pub?gid=0&single=true&output=csv'
LINK_LOCALITIES = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSKbKoMl477qDAiyhj9sT9TZlGC7OGzShbDAaELXgkPSR95_CkmZsWmN3vLGsJgEBDkiLdn4Be_mkBJ/pub?gid=121203183&single=true&output=csv'

urllib.request.urlretrieve(LINK_ACTIVITIES, PATH_DATA + 'activities.csv')
urllib.request.urlretrieve(LINK_LOCALITIES, PATH_DATA + 'localities.csv')


('/home/casus-fwu036/projects/itinerary-analysis_premysl-otakar/src/processing/../../data/01_raw/localities.csv',
 <http.client.HTTPMessage at 0x7f567b0f3b20>)

In [9]:
# download and merge srtm 

LINK_SRTM1 = 'https://edcintl.cr.usgs.gov/downloads/sciweb1/shared/topo/downloads/GMTED/Global_tiles_GMTED/075darcsec/med/E000/30N000E_20101117_gmted_med075.tif'
LINK_SRTM2 = 'https://edcintl.cr.usgs.gov/downloads/sciweb1/shared/topo/downloads/GMTED/Global_tiles_GMTED/075darcsec/med/E000/50N000E_20101117_gmted_med075.tif'


urllib.request.urlretrieve(LINK_SRTM1, DIR_OUT_SRTM + 'srtm1.tif')
urllib.request.urlretrieve(LINK_SRTM2, DIR_OUT_SRTM + 'srtm2.tif')

('/home/casus-fwu036/projects/itinerary-analysis_premysl-otakar/src/processing/../../data/01_raw//srtm/srtm2.tif',
 <http.client.HTTPMessage at 0x7f567b0f3970>)

In [11]:
# merge srtm files
source_srtm = [DIR_OUT_SRTM + file for file in os.listdir(DIR_OUT_SRTM)]

# merge srtm files
merge_command = ["gdal_merge.py", "-o", DIR_OUT_SRTM + "merged.tif"] + source_srtm
subprocess.check_output(merge_command)

b'0...10...20...30...40...50...60...70...80...90...100 - done.\n'

In [ ]:
# download rivers
LINK_RIVERS_ALL = "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/physical/ne_10m_rivers_lake_centerlines.zip"
LINK_RIVERS_EU = "https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/physical/ne_10m_lakes_europe.zip"

urllib.request.urlretrieve(LINK_RIVERS_ALL, DIR_OUT_RIVERS + 'rivers_all.zip')
urllib.request.urlretrieve(LINK_RIVERS_EU, DIR_OUT_RIVERS + 'rivers_eu.zip')

In [42]:
# process rivers

rivers_all = gpd.read_file(DIR_OUT_RIVERS + 'rivers_all.zip')
rivers_eu = gpd.read_file(DIR_OUT_RIVERS + 'rivers_eu.zip')

rivers_df = pd.concat([rivers_all, rivers_eu])
rivers_gdf = gpd.GeoDataFrame(rivers_df, geometry='geometry').set_geometry('geometry')

rivers_gdf[['geometry']].to_file(DIR_OUT_RIVERS + 'rivers.geojson', driver='GeoJSON')